#### **Gruppo 5.b**: Caponi Marco (matricola: 508773) - Ceneda Gianluca (matricola: 488257)

# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: boundingbox

Variabili utili per testare il funzionamento


In [1]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads
using Test
using DataStructures


In [22]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);                                          #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);                               #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];                                                   #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Versione iniziale di boundingbox

La funzione boundingbox serve a creare il bounding Box di una cella, cioè la scatola di misura più piccola (area, volume, ipervolume) entro cui sono contenuti tutti i punti.

In [3]:
function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

boundingbox (generic function with 1 method)

In [17]:
@btime boundingbox(V)   #21,032 μs

  21.032 μs (63 allocations: 2.19 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])

In [ ]:
@code_warntype boundingbox(V)

Attraverso la macro _@code_warntype_ abbiamo visto che boundingbox è type unstable

### versione parallelizzata boundingbox

la versione iniziale era type unstable ed era dovuto unicamente alla funzione mapslices. Per risolvere questo problema abbiamo scritto da capo questa funzione utilizzando la macro _@threads.@spawn_.
In questo modo si è ottenuta la stabilità di tipo e un miglioramento delle prestazioni

In [6]:
function boundingboxMOD(vertices::Lar.Points)
    firstDim = vertices[1,:]
    secondDim = vertices[2,:]
     if (size(vertices,1)==3)
        thirdDim = vertices[3,:]
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...), min(thirdDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...), max(thirdDim...)])
    else
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...)])
    end
    return fetch(minimum),fetch(maximum)
 end

boundingboxMOD (generic function with 1 method)

In [16]:
@btime boundingboxMOD(V)   #14.583 μs

  14.583 μs (147 allocations: 5.03 KiB)


([0.0; 0.0;;], [2.0; 1.0;;])

## TEST

In [13]:
using Test

@testset "boundingbox Tests" begin

	# 2x2x2 cuboidal grid for 1-, 2-, and 3-dim tests
	V,(VV,EV,FV,CV) = Lar.cuboidGrid([2,2,2],true)
	W,_ = Lar.apply(Lar.r(1,1,pi/6),(V,[VV,EV,FV,CV]))

	function test_bboxes(bboxes)
		# initialize accumulator
		accumulator = BitArray{1}()
		for k=1:size(bboxes[1],1)
			push!(accumulator, true)
		end
		# testing data 
		for h=1:length(bboxes)
			accumulator = (bboxes[h][:,1] .< bboxes[h][:,2]) .& accumulator
		end
		return (&)(accumulator...)
	end
	
	@testset "boundingbox Tests" begin
			
		@testset "Edge tests" begin # 
			cellpoints = [ W[:,EV[k]]::Lar.Points for k=1:length(EV) ]
			bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
			@test true == test_bboxes(bboxes)
		end
		@testset "Face tests" begin # 
			cellpoints = [ W[:,FV[k]]::Lar.Points for k=1:length(FV) ]
			bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
			@test true == test_bboxes(bboxes)
		end
		@testset "Cell tests" begin # 
			cellpoints = [ W[:,CV[k]]::Lar.Points for k=1:length(CV) ]
			bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
			@test true == test_bboxes(bboxes)
		end
	end
end

Test Summary:     | Pass  Total
boundingbox Tests |    3      3


Test.DefaultTestSet("boundingbox Tests", Any[Test.DefaultTestSet("boundingbox Tests", Any[Test.DefaultTestSet("Edge tests", Any[], 1, false, false), Test.DefaultTestSet("Face tests", Any[], 1, false, false), Test.DefaultTestSet("Cell tests", Any[], 1, false, false)], 0, false, false)], 0, false, false)

![test di boundingbox](https://github.com/MarcoCap13/LARSplitting2D/blob/main/docs/test/boundingbox_test.png?raw=true)

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [20]:
@benchmark boundingbox(V)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  21.069 μs …   8.234 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     22.031 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   49.522 μs ± 225.380 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅▂▁▁                                                        ▁
  ███████▇▇▆▆▆▆▆▆▆▅▆▆▆▆▅▅▅▅▅▅▄▄▄▅▅▅▄▅▃▅▄▄▅▄▄▁▄▄▅▁▅▁▃▄▃▁▅▄▅▃▃▄▄ █
  21.1 μs       Histogram: log(frequency) by time       659 μs <

 Memory estimate: 2.19 KiB, allocs estimate: 63.

funzione modificata:

In [19]:
@benchmark boundingboxMOD(V)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  14.511 μs …  11.761 ms  ┊ GC (min … max): 0.00% … 99.09%
 Time  (median):     15.507 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.674 μs ± 118.120 μs  ┊ GC (mean ± σ):  5.92% ±  0.99%

  █▇▃▂             ▁                                           ▁
  ██████▇▇▇▇▆▅▆▇▇▇████▇▇▆▆▆▆▅▅▄▄▅▅▅▅▅▅▅▄▅▄▄▃▅▅▅▃▅▄▂▃▅▃▅▄▃▃▂▄▄▄ █
  14.5 μs       Histogram: log(frequency) by time      81.3 μs <

 Memory estimate: 5.03 KiB, allocs estimate: 147.